In [ ]:
from huggingface_hub import login, snapshot_download
import datasets
import os as os
def load_gaia_dataset(use_raw_dataset: bool, set_to_run: str) -> datasets.Dataset:
    if not os.path.exists("gaia"):
        if use_raw_dataset:
            snapshot_download(
                repo_id="gaia-benchmark/GAIA",
                repo_type="dataset",
                local_dir="gaia",
                ignore_patterns=[".gitattributes", "README.md"],
            )
        else:
            # WARNING: this dataset is gated: make sure you visit the repo to require access.
            snapshot_download(
                repo_id="smolagents/GAIA-annotated",
                repo_type="dataset",
                local_dir="gaia",
                ignore_patterns=[".gitattributes", "README.md"],
            )

    def preprocess_file_paths(row):
        if len(row["file_name"]) > 0:
            row["file_name"] = f"gaia/{set_to_run}/" + row["file_name"]
        return row

    eval_ds = datasets.load_dataset(
        "gaia-benchmark/GAIA",
        name="2023_level1",
        split=set_to_run,
        # data_files={"validation": "validation/metadata.jsonl", "test": "test/metadata.jsonl"},
    )

    eval_ds = eval_ds.rename_columns({"Question": "question", "Final answer": "true_answer", "Level": "task"})
    eval_ds = eval_ds.map(preprocess_file_paths)
    return eval_ds


In [ ]:
q = load_gaia_dataset(use_raw_dataset=False, set_to_run="test")


In [ ]:
import pandas as pd
print("Loaded evaluation dataset:")
print(pd.DataFrame(q)["task"].value_counts())


In [ ]:
# q = load_gaia_dataset(use_raw_dataset=False, set_to_run="validation")
from datasets import load_dataset
load_dotenv()
dataset = load_dataset("gaia-benchmark/GAIA", '2023_level2')
print(dataset)
